## Voice Deepfake - Real-time Voice Style Transfer with CNNs

## Requirements
TensorFlow needs to be installed before running the training script. Code is tested on TensorFlow version 1.0.1 for Python 2.7 and Python 3.5.

In addition, librosa must be installed for reading and writing audio.

To install the required python packages, run

pip install -r requirements.txt
For GPU support, use

pip install -r requirements_gpu.txt

#### Inspiration
https://www.ted.com/talks/holly_herndon_what_if_you_could_sing_in_your_favorite_musician_s_voice?language=en

#### Resources
https://becominghuman.ai/convoice-real-time-zero-shot-voice-style-transfer-with-convolutional-network-4c7b7fff66c9

#### Network Architecture:

Connectionist temporal classification loss (CTC loss)
